<a href="https://colab.research.google.com/github/Marsobad/ADL/blob/master/ELTA_project_combined.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import re
import spacy
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
from sklearn import tree
from sklearn.ensemble import BaggingClassifier,RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, truncnorm, randint
from pprint import pprint
#from google.colab import drive
#drive.mount('/content/drive')

Data loading

In [0]:
x_path = "/content/drive/My Drive/ELTA/X_train_update.csv"
y_path = "/content/drive/My Drive/ELTA/Y_train_CVw08PX.csv"
#x_df = pd.read_csv(x_path)
y_df = pd.read_csv(y_path)
# drop useless columns
#x_df.drop(columns=['Unnamed: 0', 'description', 'productid', 'imageid'], axis=0, inplace=True)

### Tokenization
At first we clean the accent using the function normalize accent

In [0]:
def normalize_accent(string):
    string = string.replace('á', 'a')
    string = string.replace('â', 'a')

    string = string.replace('é', 'e')
    string = string.replace('è', 'e')
    string = string.replace('ê', 'e')
    string = string.replace('ë', 'e')

    string = string.replace('î', 'i')
    string = string.replace('ï', 'i')

    string = string.replace('ö', 'o')
    string = string.replace('ô', 'o')
    string = string.replace('ò', 'o')
    string = string.replace('ó', 'o')

    string = string.replace('ù', 'u')
    string = string.replace('û', 'u')
    string = string.replace('ü', 'u')

    string = string.replace('ç', 'c')
    string = re.sub(r'°|#|/|-|%|_|[0-9]+', '', string)
    
    return string

def raw_to_tokens(raw_string, spacy_nlp):
    string = raw_string.lower().rstrip()
    string = normalize_accent(string)
    spacy_tokens = spacy_nlp(string)
    string_tokens = [token.orth_ for token in spacy_tokens if not token.is_punct if not token.is_stop]
    clean_string = " ".join(string_tokens)
    return clean_string

#spacy_nlp = spacy.load("fr")
#x_df['tokens'] = x_df['designation'].apply(lambda s : raw_to_tokens(s,spacy_nlp))
#x_df.to_csv('./tokens.csv')
x_df = pd.read_csv('/content/drive/My Drive/ELTA/tokens.csv')
x_df.tail()
#y_df.prdtypecode.value_counts()
x_df, y_df = x_df[ ~ x_df.tokens.isna()], y_df[~ x_df.tokens.isna()]
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(x_df['tokens'])
y = y_df.prdtypecode
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [0]:
weighted_f1_scores = {}

##Decision tree
Weigthed f1 score on test set : 0.69


In [0]:
clf = tree.DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
weighted_f1_scores["decision tree"] = f1_score(y_test, y_pred,average="weighted")

## Bagging
Weighted f1 score : 0.70

In [0]:
clf_bag = BaggingClassifier(tree.DecisionTreeClassifier(), max_samples=0.5)
clf_bag.fit(X_train, y_train)
y_pred_bag = clf_bag.predict(X_test)
weighted_f1_scores["bagging"]  = f1_score(y_test, y_pred_bag,average="weighted")

## Random forest
F1 weigted score : 0.73 (without grid search)


In [0]:
model_params = {
    'n_estimators': randint(4,200),
    'max_features': truncnorm(a=0, b=1, loc=0.25, scale=0.1),
    'min_samples_split': uniform(0.01, 0.199)
}

In [0]:
rf_model = RandomForestClassifier()
clf_rf = RandomizedSearchCV(rf_model, model_params, n_iter=10, cv=5, random_state=1,n_jobs = -1)
model = clf_rf.fit(X_train,y_train)

KeyboardInterrupt: ignored

In [0]:
pprint(model.best_estimator_.get_params())

In [0]:
y_pred_rf = clf_rf.predict(X_test)
weighted_f1_scores["random forest classifier"]  = f1_score(y_test, y_pred_rf,average="weighted")

##Adaboost classifier 

In [0]:
estimators = 70
alpha = 0.01
random = None
model = AdaBoostClassifier(n_estimators = estimators, learning_rate = alpha, random_state = random )

In [0]:
res = model.fit(X_train, y_train)

In [0]:
model.score(X_train, y_train)

In [0]:
y_pred = model.predict(X_test)
f1_score(y_test, y_pred,average="weighted")

In [0]:
param_dist = {
 'n_estimators': randint(100,200),
 'learning_rate' : [0.1,0.3,1] }
pre_gs_inst = RandomizedSearchCV(AdaBoostClassifier(),
 param_distributions = param_dist,
 cv=3,
 n_iter = 5,
 n_jobs=-1)

In [0]:
pre_gs_inst.fit(X_train, y_train)

KeyboardInterrupt: ignored